# 0. Libraries

In [1]:
import pandas as pd
import os
import glob
import re

# 1. Loading the data

In [3]:
# Define function to extract state code from filename
def extract_state_code(filename):
    # Extract the 2-letter state code at the end of the filename
    match = re.search(r'_([A-Z]{2})\.csv$', filename)
    if match:
        return match.group(1)
    return None

In [33]:
# Example of a correctly formatted .csv file of Google Trends data
sample = pd.read_csv('../google_trends_data/issue_healthcare/health/health_AK.csv', skiprows=1)
print("Sample of a correctly formatted .csv file of Google Trends data:")
display(sample.head())

# Example of a badly formatted .csv file of Google Trends data
sample2 = pd.read_csv('../google_trends_data/issue_guns_and_criminal_justice/criminal justice/criminal justice_PA.csv', skiprows=1)
print("Sample of a badly formatted .csv file of Google Trends data:")
display(sample2.head())

Sample of a correctly formatted .csv file of Google Trends data:


,Mes,health: (Alaska)
0,2015-01,71
1,2015-02,77
2,2015-03,75
3,2015-04,76
4,2015-05,59


Sample of a badly formatted .csv file of Google Trends data:


,"criminal justice: (1/1/15 - 10/5/25, Pensilvania)"


In [34]:
# Initialize empty list to store all data
all_data = []

# Root directory containing all issue folders
root_dir = '../google_trends_data/'  # Update this to the appropriate path if needed

# Get all issue folders (they must start with 'issue_')
issue_folders = [f for f in os.listdir(root_dir) if f.startswith('issue_') and os.path.isdir(os.path.join(root_dir, f))]

# Traverse through each issue folder and its subfolders
for issue_folder in issue_folders:
    issue_path = os.path.join(root_dir, issue_folder)
    
    # Get term folders (subfolders) within each issue folder
    term_folders = [f for f in os.listdir(issue_path) if os.path.isdir(os.path.join(issue_path, f))]
    
    # Traverse through each term folder (e.g., 'health' in the 'issue_healthcare' folder)
    for term_folder in term_folders:
        term_path = os.path.join(issue_path, term_folder)
        term_name = term_folder  # The folder name represents the term
        
        # Get all CSV files within the term folder
        csv_files = glob.glob(os.path.join(term_path, '*.csv'))  # This will create a list of all CSV files in the term folder
        print(f"Processing {len(csv_files)} CSV files in {term_path}...")

        for csv_file in csv_files:
            # Extract state code from filename
            state_code = extract_state_code(csv_file)
            if state_code:  # If state code is found, try to read the CSV
                try:
                    # Read the CSV file
                    df = pd.read_csv(csv_file, skiprows=1)  # Skip the first row (it contains unwanted info)
                    
                    # Each CSV has two columns: month and value (the index)
                    # Rename columns for clarity
                    if len(df.columns) >= 2:
                        df.columns = ['month', 'value'] + list(df.columns[2:])
                        
                        # Add state and term info to the dataframe
                        df['state'] = state_code
                        df['term'] = term_name
                        
                        # Append to our list
                        all_data.append(df[['month', 'state', 'term', 'value']])
                    
                    elif len(df.columns) == 1:
                        print(f"No data found in {csv_file} (empty .csv)")

                    else:
                        print(f"Unexpected format in {csv_file}: {df.columns}")
                except Exception as e:
                    print(f"Error processing {csv_file}: {e}")
            else:
                print(f"Could not extract state code from {csv_file}")
        print(f"Finished processing {len(csv_files)} CSV files in {term_path}\n")

# Combine all data into one dataframe
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Pivot the dataframe to get the desired structure
    # Each row is a month-state combination, columns are terms
    pivoted_df = combined_df.pivot_table(
        index=['month', 'state'], 
        columns='term', 
        values='value', 
        aggfunc='first'
    ).reset_index()
    
    # Sort by state and month
    pivoted_df = pivoted_df.sort_values(by=['state', 'month']).reset_index(drop=True)

    # Display the result
    display(pivoted_df.head())

else:
    print("No data found!")

Processing 51 CSV files in ../google_trends_data/issue_guns_and_criminal_justice/criminal justice...
Finished processing 51 CSV files in ../google_trends_data/issue_guns_and_criminal_justice/criminal justice

Processing 35 CSV files in ../google_trends_data/issue_guns_and_criminal_justice/guns...
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/gns_CT.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_AR.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_DE.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_IN.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_KS.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_KY.csv (empty .csv)
No data found in ../google_trends_data/issue_guns_and_criminal_justice/guns/guns_LA.csv (empty .c

term,month,state,criminal justice,guns,health,healthcare,violence schools gun
0,2015-01,AK,53,94,71,40,NaN
1,2015-02,AK,0,84,77,50,NaN
2,2015-03,AK,55,95,75,47,NaN
3,2015-04,AK,0,77,76,40,NaN
4,2015-05,AK,0,76,59,33,NaN


In [25]:
pivoted_df.head()

term,month,state,criminal justice,guns,health,healthcare,violence schools gun
0,2015-01,AK,53,94,71,40,NaN
1,2015-02,AK,0,84,77,50,NaN
2,2015-03,AK,55,95,75,47,NaN
3,2015-04,AK,0,77,76,40,NaN
4,2015-05,AK,0,76,59,33,NaN


After manually checking some of the results from the data frame, everything seems to make sense and to match with the source Google Trends files. 